## Modelo Supervisionado 

**Projeto**: Expansão por grupos — classificador de “Bom Desempenho” por região×grupo
**Objetivo**: Treinar e avaliar um RandomForest com pipeline (pré-processamento + OHE), escolher limiar ótimo por F1, salvar o modelo e ranquear candidatos.

In [ ]:
# # Notebook 1: Configurações Iniciais
# Configurações globais, importações e parâmetros do projeto.
# Execute este notebook primeiro para preparar o ambiente.

# ===== Imports principais =====
import glob
from pathlib import Path
import warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns  # Adicionado para visualizações mais ricas
from sklearn.model_selection import StratifiedKFold, StratifiedShuffleSplit, GridSearchCV, RandomizedSearchCV
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler  # Adicionado StandardScaler para normalização
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import (classification_report, confusion_matrix, roc_auc_score,
                             average_precision_score, precision_recall_curve, roc_curve, accuracy_score, f1_score)
from sklearn.inspection import permutation_importance
from joblib import dump, load
import logging  # Adicionado para logs
import seaborn as sns
# ===== Configurações globais =====
warnings.filterwarnings("ignore", category=UserWarning)
pd.set_option("display.max_columns", None)
pd.set_option("display.width", 220)
pd.set_option("display.max_rows", 200)

sns.set_style("whitegrid")  # Estilo moderno do seabornlogging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# ===== Parâmetros do projeto =====
CAMINHO_DF_HIST = "../../../database/dataset gerado/dataset_limpo.csv"
ARQ_MODELO = Path("../../../database/dataset gerado/modelo_arvore_decisao_perfeito.joblib")
ARQ_NOVAS = "../../../database/dataset gerado/sugestoes_expansao_para_supervisionado"
TOP_N = 10
TOP_K = 1
FORCAR_TREINO = True
LIMIAR = None
RANDOM_STATE = 42
N_JOBS = -1  # Paralelização total

# ===== Função utilitária para leitura de CSV =====
def read_csv_flex(base_path_or_stem: str) -> pd.DataFrame:
    """Lê CSV de forma flexível, testando múltiplos caminhos e separadores."""
    candidates = [
        Path(base_path_or_stem),
        Path(f"{base_path_or_stem}.csv"),
        Path("data") / f"{base_path_or_stem}.csv",
        Path("datasets") / f"{base_path_or_stem}.csv",
    ]
    for p in glob.glob(f"**/{Path(base_path_or_stem).stem}*.csv", recursive=True):
        candidates.append(Path(p))
    
    tried = []
    for path in candidates:
        if not path.exists():
            tried.append(str(path))
            continue
        for sep in [None, ';', ',']:
            try:
                df = pd.read_csv(path, sep=sep, engine="python")
                logging.info(f"Arquivo carregado: {path}")
                return df
            except Exception as e:
                tried.append(f"{path}(sep={sep}): {str(e)}")
    raise FileNotFoundError("CSV não encontrado. Tentativas: " + " | ".join(tried))

In [ ]:
# # Notebook 2: Preparação de Dados
# Constrói a base de treino com features otimizadas e labels.
# Execute após o Notebook 1.

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from sklearn.preprocessing import StandardScaler

# Função para construir a tabela de treino
def build_train_table(df_hist: pd.DataFrame, top_k: int = 1):
    """Constrói a tabela de treino com features otimizadas e rótulos."""
    need = ["ID_Loja", "Desconto", "Total_Preco_Varejo", "Total_Preco_Liquido",
            "Dim_Lojas.REGIAO_CHILLI", "Dim_Produtos.GRUPO_CHILLI", "Dim_Produtos.Grupo_Produto"]
    miss = [c for c in need if c not in df_hist.columns]
    if miss:
        raise ValueError(f"Faltando colunas no histórico: {miss}")

    # Tratamento de outliers (limite de 3 desvios padrão para variáveis numéricas)
    for col in ["Total_Preco_Varejo", "Total_Preco_Liquido", "Desconto"]:
        if col in df_hist:
            mean, std = df_hist[col].mean(), df_hist[col].std()
            df_hist[col] = df_hist[col].clip(lower=mean - 3 * std, upper=mean + 3 * std)

    # Agregações por região
    reg_lojas = df_hist.groupby("Dim_Lojas.REGIAO_CHILLI")["ID_Loja"].nunique()
    reg_receita = df_hist.groupby("Dim_Lojas.REGIAO_CHILLI")["Total_Preco_Liquido"].sum()
    reg_receita_med = df_hist.groupby("Dim_Lojas.REGIAO_CHILLI")["Total_Preco_Liquido"].median()
    reg_receita_p75 = df_hist.groupby("Dim_Lojas.REGIAO_CHILLI")["Total_Preco_Liquido"].quantile(0.75)  # Percentil 75
    agg_reg = pd.concat([reg_receita, reg_lojas, reg_receita_med, reg_receita_p75], axis=1).reset_index()
    agg_reg.columns = ["Dim_Lojas.REGIAO_CHILLI", "regiao_receita_total", "regiao_num_lojas",
                       "regiao_receita_mediana", "regiao_receita_p75"]
    agg_reg["regiao_receita_por_loja"] = agg_reg["regiao_receita_total"] / agg_reg["regiao_num_lojas"].replace(0, np.nan).fillna(0)

    # Medianas e variâncias por região x grupo
    grp_meds = df_hist.groupby(["Dim_Lojas.REGIAO_CHILLI", "Dim_Produtos.GRUPO_CHILLI"]).agg(
        grupo_med_preco_varejo=("Total_Preco_Varejo", "median"),
        grupo_med_desconto=("Desconto", "median"),
        grupo_var_preco_varejo=("Total_Preco_Varejo", "var"),
        grupo_p90_preco_varejo=("Total_Preco_Varejo", lambda x: np.percentile(x, 90))  # Percentil 90
    ).reset_index()

    # Volume por região x grupo
    vol = df_hist.groupby(["Dim_Lojas.REGIAO_CHILLI", "Dim_Produtos.GRUPO_CHILLI"])["Total_Preco_Liquido"].sum().reset_index()
    vol.columns = ["Dim_Lojas.REGIAO_CHILLI", "Dim_Produtos.GRUPO_CHILLI", "grupo_regiao_receita"]
    vol["rank_na_regiao"] = vol.groupby("Dim_Lojas.REGIAO_CHILLI")["grupo_regiao_receita"].rank(method="first", ascending=False)
    vol["Bom_Desempenho"] = (vol["rank_na_regiao"] <= top_k).astype(int)

    # Montagem de X e y
    X = vol[["Dim_Lojas.REGIAO_CHILLI", "Dim_Produtos.GRUPO_CHILLI"]].merge(
        agg_reg, on="Dim_Lojas.REGIAO_CHILLI", how="left"
    ).merge(
        grp_meds, on=["Dim_Lojas.REGIAO_CHILLI", "Dim_Produtos.GRUPO_CHILLI"], how="left"
    )
    y = vol["Bom_Desempenho"].astype(int)

    # Normalização das variáveis numéricas
    num_cols = ["regiao_receita_total", "regiao_num_lojas", "regiao_receita_por_loja",
                "regiao_receita_mediana", "regiao_receita_p75", "grupo_med_preco_varejo",
                "grupo_med_desconto", "grupo_var_preco_varejo", "grupo_p90_preco_varejo"]
    scaler = StandardScaler()
    X[num_cols] = scaler.fit_transform(X[num_cols].fillna(0))

    # Catálogo por grupo
    catalogo = df_hist.groupby(["Dim_Produtos.GRUPO_CHILLI", "Dim_Produtos.Grupo_Produto"]).agg(
        Preco_Varejo_med=("Total_Preco_Varejo", "median")
    ).reset_index()

    return X, y, catalogo, agg_reg, grp_meds

# Carregamento do histórico
dfh = read_csv_flex(CAMINHO_DF_HIST)
dfh = dfh.drop(columns=[
    "ID_Cliente", "Dim_Cliente.Data_Nascimento", "Dim_Cliente.Regiao_Cliente",
    "ID_Produto", "Dim_Lojas.Nome_Emp", "Dim_Lojas.Bairro_Emp", "Dim_Lojas.Cidade_Emp",
    "Dim_Lojas.CANAL_VENDA", "Dim_Lojas.Tipo_PDV", "Dim_Lojas.Regiao"
], errors="ignore")

# Construção da base
X_all, y_all, catalogo, agg_reg, grp_meds = build_train_table(dfh, TOP_K)
logging.info(f"Forma de X: {X_all.shape} | Taxa de positivos: {y_all.mean().round(3)}")

# Visualização da distribuição do alvo
plt.figure(figsize=(6, 4))
sns.countplot(x=y_all, palette="viridis")
plt.title("Distribuição do Alvo (Bom_Desempenho)")
plt.xlabel("Classe")
plt.ylabel("Contagem")
plt.show()


In [ ]:
# # Notebook 3: Definição e Otimização do Modelo
# Cria e otimiza o modelo de Árvore de Decisão com GridSearchCV.

from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.metrics import accuracy_score, average_precision_score, roc_auc_score, f1_score
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import StratifiedKFold
import numpy as np
import pandas as pd
import logging

# Função para criar o pré-processador
def make_preprocessor(X):
    """Cria o pré-processador para transformar os dados."""
    cat_cols = [c for c in ["Dim_Produtos.GRUPO_CHILLI", "Dim_Lojas.REGIAO_CHILLI"] if c in X.columns]
    num_cols = ["regiao_receita_total", "regiao_num_lojas", "regiao_receita_por_loja",
                "regiao_receita_mediana", "regiao_receita_p75", "grupo_med_preco_varejo",
                "grupo_med_desconto", "grupo_var_preco_varejo", "grupo_p90_preco_varejo"]

    oh = OneHotEncoder(handle_unknown="ignore", sparse_output=False)
    pre = ColumnTransformer(
        transformers=[
            ("cat", Pipeline([("imputer", SimpleImputer(strategy="most_frequent")), ("onehot", oh)]), cat_cols),
            ("num", Pipeline([("imputer", SimpleImputer(strategy="median")), ("scaler", StandardScaler())]), num_cols),
        ],
        remainder="drop"
    )
    return pre, cat_cols + num_cols

# Função para criar o pipeline do modelo (sem pré-processamento)
def make_model():
    """Cria pipeline apenas com o classificador para dados pré-processados."""
    clf = DecisionTreeClassifier(class_weight="balanced", random_state=RANDOM_STATE)
    pipeline = Pipeline([("clf", clf)])

    param_grid = {
        'clf__max_depth': [2, 3, 5, 7],  # Profundidades menores para evitar overfitting
        'clf__min_samples_leaf': [20, 50, 100],  # Mais amostras por folha
        'clf__min_samples_split': [20, 50, 100],  # Mais amostras para split
        'clf__criterion': ['gini', 'entropy'],
        'clf__max_features': ['sqrt', 'log2']
    }

    cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=RANDOM_STATE)
    search = GridSearchCV(pipeline, param_grid=param_grid, cv=cv, scoring='average_precision', n_jobs=N_JOBS)
    return search

# Função para validação cruzada com dados pré-processados
def crossval_scores(model, X, y):
    """Executa validação cruzada e retorna métricas detalhadas."""
    model.fit(X, y)
    logging.info(f"Melhores parâmetros: {model.best_params_}")
    aps, accs, aucs, f1s = [], [], [], []
    cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=RANDOM_STATE)
    for tr, te in cv.split(X, y):
        m = Pipeline(model.best_estimator_.steps)
        m.fit(X[tr], y[tr])
        proba = m.predict_proba(X[te])[:, 1]
        pred = (proba >= 0.5).astype(int)
        aps.append(average_precision_score(y[te], proba))
        accs.append(accuracy_score(y[te], pred))
        aucs.append(roc_auc_score(y[te], proba))
        f1s.append(f1_score(y[te], pred))
    logging.info(f"CV (DT Otimizada) AP={np.mean(aps):.3f} ± {np.std(aps):.3f} | "
                 f"ACC={np.mean(accs):.3f} ± {np.std(accs):.3f} | "
                 f"ROC-AUC={np.mean(aucs):.3f} ± {np.std(aucs):.3f} | "
                 f"F1={np.mean(f1s):.3f} ± {np.std(f1s):.3f}")

# Criação do pré-processador
preprocessor, feature_names = make_preprocessor(X_all)

# Pré-processamento antes do SMOTE
X_all_transformed = preprocessor.fit_transform(X_all)
logging.info(f"Forma após pré-processamento: {X_all_transformed.shape}")

# Aplicar SMOTE
smote = SMOTE(random_state=RANDOM_STATE)
X_all_balanced, y_all_balanced = smote.fit_resample(X_all_transformed, y_all)
logging.info(f"Proporção após SMOTE: {y_all_balanced.mean():.3f}")

# Criação e avaliação do modelo com dados balanceados
model = make_model()
crossval_scores(model, X_all_balanced, y_all_balanced)

# Salvar variáveis globais para uso nos notebooks subsequentes
globals()['X_all_balanced'] = X_all_balanced
globals()['y_all_balanced'] = y_all_balanced
globals()['preprocessor'] = preprocessor
globals()['feature_names'] = feature_names
logging.info("Variáveis globais salvas: X_all_balanced, y_all_balanced, preprocessor, feature_names")


In [ ]:

# # Notebook 4: Seleção de Limiar e Visualizações
# Escolhe o limiar ótimo e gera visualizações.

import seaborn as sns
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, precision_recall_curve, roc_curve, roc_auc_score, average_precision_score
from sklearn.model_selection import StratifiedShuffleSplit
import numpy as np
import matplotlib.pyplot as plt
import logging

# Função para escolher limiar e gerar plots
def choose_threshold_and_plots(model, X, y, limiar=None):
    """Seleciona limiar ótimo e gera visualizações."""
    if limiar is not None:
        return float(limiar)
    sss = StratifiedShuffleSplit(n_splits=1, test_size=0.4, random_state=RANDOM_STATE)  # Aumentado para 0.4
    tr, te = next(sss.split(X, y))
    model.fit(X[tr], y[tr])  # Dados já pré-processados
    proba = model.predict_proba(X[te])[:, 1]
    yte = y[te]

    logging.info("\n=== Hold-out (estratificado) — limiar 0.50 ===")
    pred050 = (proba >= 0.5).astype(int)
    acc_050 = accuracy_score(yte, pred050)
    print(f"Acurácia (limiar 0.50): {acc_050:.3f}")
    print(classification_report(yte, pred050, digits=3))
    print("Matriz:\n", confusion_matrix(yte, pred050))
    print(f"ROC-AUC: {roc_auc_score(yte, proba):.3f} | PR-AUC: {average_precision_score(yte, proba):.3f}")

    prec, rec, thr = precision_recall_curve(yte, proba)
    f1s = 2 * (prec * rec) / (prec + rec + 1e-9)
    idx = np.nanargmax(f1s)
    chosen = float(thr[idx]) if idx < len(thr) else 0.5
    logging.info(f"\n→ Limiar ótimo (F1 máx): {chosen:.3f}")

    pred_opt = (proba >= chosen).astype(int)
    acc_opt = accuracy_score(yte, pred_opt)
    print(f"Acurácia (limiar ótimo {chosen:.3f}): {acc_opt:.3f}")

    plt.figure(figsize=(6, 4))
    sns.lineplot(x=rec, y=prec, label="Curva PR")
    plt.scatter(rec[idx], prec[idx], color='red', label="F1 Máximo")
    plt.title("Curva Precision–Recall (Hold-out)")
    plt.xlabel("Recall")
    plt.ylabel("Precision")
    plt.legend()
    plt.show()

    plt.figure(figsize=(6, 4))
    xs = thr if len(thr) else np.array([0.5])
    ys = f1s[:-1] if len(thr) else np.array([0.0])
    sns.lineplot(x=xs, y=ys, label="F1 Score")
    if len(thr):
        plt.axvline(chosen, linestyle="--", color='red', label="Limiar Ótimo")
    plt.title("F1 × Limiar (Hold-out)")
    plt.xlabel("Limiar")
    plt.ylabel("F1")
    plt.legend()
    plt.show()

    fpr, tpr, _ = roc_curve(yte, proba)
    plt.figure(figsize=(6, 4))
    sns.lineplot(x=fpr, y=tpr, label="Curva ROC")
    plt.plot([0, 1], [0, 1], linestyle="--", color='gray', label="Aleatório")
    plt.title("Curva ROC (Hold-out)")
    plt.xlabel("FPR")
    plt.ylabel("TPR")
    plt.legend()
    plt.show()

    cm = confusion_matrix(yte, pred_opt)
    plt.figure(figsize=(5, 4))
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", cbar=False)
    plt.title("Matriz de Confusão (Limiar Ótimo)")
    plt.xticks([0.5, 1.5], ["Neg", "Pos"])
    plt.yticks([0.5, 1.5], ["Neg", "Pos"])
    plt.xlabel("Predito")
    plt.ylabel("Verdadeiro")
    plt.show()

    return chosen

# Execução
chosen_thr = choose_threshold_and_plots(model, X_all_balanced, y_all_balanced, LIMIAR)
globals()['chosen_thr'] = chosen_thr  # Salvar para notebooks subsequentes
logging.info(f"Limiar escolhido salvo: {chosen_thr:.3f}")


In [ ]:
# # Notebook 5: Treino Final e Importância de Features
# Realiza o treino final, salva o modelo e analisa importância de features.

from joblib import dump
from sklearn.inspection import permutation_importance
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import logging

# Verificar se as variáveis globais estão disponíveis
if 'X_all_balanced' not in globals() or 'y_all_balanced' not in globals() or 'preprocessor' not in globals():
    raise ValueError("Variáveis globais X_all_balanced, y_all_balanced ou preprocessor não encontradas. Execute o Notebook 3 primeiro.")

# Treino final com dados pré-processados e balanceados
model.fit(X_all_balanced, y_all_balanced)
dump(model, ARQ_MODELO.as_posix())
logging.info(f"Modelo salvo em: {ARQ_MODELO.name}")

# Importância por permutação
feature_names = preprocessor.get_feature_names_out()  # Nomes das colunas transformadas
result = permutation_importance(model, X_all_balanced, y_all_balanced, n_repeats=10, random_state=RANDOM_STATE, n_jobs=N_JOBS)
importances = pd.DataFrame({
    "feature": feature_names,
    "importance_mean": result.importances_mean,
    "importance_std": result.importances_std
}).sort_values("importance_mean", ascending=False)
display(importances.head(15))

# Visualização Top-10 importâncias
top = importances.head(10).iloc[::-1]
plt.figure(figsize=(7, 5))
sns.barplot(x="importance_mean", y="feature", data=top, palette="viridis")
plt.title("Importância por Permutação — Top 10")
plt.xlabel("Δ Métrica (Média)")
plt.ylabel("Feature")
plt.tight_layout()
plt.show()


In [ ]:
# # Notebook 6: Predições em Novas Localizações e Ranking
# Aplica o modelo em novas localizações e exporta o ranking.

import json
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import logging

# Verificar se as variáveis globais estão disponíveis
if 'preprocessor' not in globals() or 'model' not in globals() or 'chosen_thr' not in globals():
    raise ValueError("Variáveis globais preprocessor, model ou chosen_thr não encontradas. Execute os Notebooks 3 e 4 primeiro.")

# Leitura de novas localizações
novas_raw = read_csv_flex(ARQ_NOVAS)
logging.info(f"Colunas nas novas localizações: {list(novas_raw.columns)}")

rename_map = {c: "Dim_Lojas.Estado_Emp" if c.strip().lower() == "estado" else
              "Dim_Lojas.REGIAO_CHILLI" if c.strip().lower() == "regiao_chilli" else c
              for c in novas_raw.columns}
novas = novas_raw.rename(columns=rename_map).copy()
for c in ["escopo", "cluster_id"]:
    if c not in novas.columns:
        novas[c] = "NA"

# Criação de candidatos
base = novas.rename(columns={"Dim_Lojas.REGIAO_CHILLI": "REG_TMP"}, errors="ignore").copy()
cand = base.merge(
    catalogo[["Dim_Produtos.GRUPO_CHILLI", "Dim_Produtos.Grupo_Produto", "Preco_Varejo_med"]],
    how="cross"
)

# Anexação de agregados
cand = cand.merge(agg_reg, left_on="REG_TMP", right_on="Dim_Lojas.REGIAO_CHILLI", how="left")
cand = cand.merge(grp_meds, left_on=["REG_TMP", "Dim_Produtos.GRUPO_CHILLI"],
                  right_on=["Dim_Lojas.REGIAO_CHILLI", "Dim_Produtos.GRUPO_CHILLI"], how="left",
                  suffixes=("", "_dup"))
cand["Dim_Lojas.REGIAO_CHILLI"] = cand["REG_TMP"]

# Features
feat_cols = ["Dim_Lojas.REGIAO_CHILLI", "Dim_Produtos.GRUPO_CHILLI",
             "regiao_receita_total", "regiao_num_lojas", "regiao_receita_por_loja",
             "regiao_receita_mediana", "regiao_receita_p75", "grupo_med_preco_varejo",
             "grupo_med_desconto", "grupo_var_preco_varejo", "grupo_p90_preco_varejo"]
cand_feat = cand[feat_cols].copy()

# Pré-processamento das features com o mesmo preprocessor do Notebook 3
cand_feat_transformed = preprocessor.transform(cand_feat)

# Predições
proba = model.predict_proba(cand_feat_transformed)[:, 1]
cand["Probabilidade_Sucesso"] = proba
cand["Previsao"] = (proba >= chosen_thr).astype(int)
cand["Probabilidade_pct"] = (proba * 100).round(1)

# Ranking
id_cols = ["escopo", "cluster_id"]
rank = cand.sort_values(id_cols + ["Probabilidade_Sucesso"], ascending=[True, True, False]).reset_index(drop=True)
top_n = rank.groupby(id_cols, group_keys=False).head(TOP_N)

cols_out = (id_cols + ["Dim_Lojas.REGIAO_CHILLI", "Dim_Produtos.GRUPO_CHILLI", "Dim_Produtos.Grupo_Produto",
                       "Preco_Varejo_med", "Probabilidade_pct", "Previsao"])
saida = top_n[[c for c in cols_out if c in top_n.columns]].copy()

print("\n==================== RANKING TOP-N POR LOCAL ====================")
for (esc, clu), df_loc in saida.groupby(["escopo", "cluster_id"]):
    print(f"\n>>> Local: escopo={esc} | cluster_id={clu}  (Top-{TOP_N})")
    print(df_loc.drop(columns=["escopo", "cluster_id"]).to_string(index=False))

# Visualização exemplo
first_key = next(iter(saida.groupby(["escopo", "cluster_id"]).groups.keys()))
ex = saida.set_index(["escopo", "cluster_id"]).loc[first_key].reset_index(drop=True)
plt.figure(figsize=(8, 5))
sns.barplot(x="Probabilidade_pct", y="Dim_Produtos.GRUPO_CHILLI", data=ex, palette="viridis")
plt.title(f"Top-{TOP_N} — {first_key}")
plt.xlabel("Probabilidade (%)")
plt.ylabel("Grupo")
plt.tight_layout()
plt.show()

# Exportação em CSV e JSON
ARQ_OUT = "../../../database/dataset gerado/ranking_arvore_decisao_perfeito.csv"
ARQ_OUT_JSON = "../../../database/dataset gerado/ranking_arvore_decisao_perfeito.json"
saida.to_csv(ARQ_OUT, index=False, sep=";")
saida.to_json(ARQ_OUT_JSON, orient="records", lines=True)
logging.info(f"Arquivos exportados: {ARQ_OUT}, {ARQ_OUT_JSON}")


In [ ]:
# # Notebook 7: Análise Final e Visualizações Adicionais
# Analisa o ranking exportado e realiza checagens finais.

import os
import json
import seaborn as sns

# Leitura do ranking exportado
if not os.path.exists(ARQ_OUT):
    logging.error(f"Arquivo não encontrado: {ARQ_OUT}.")
else:
    df_rank = pd.read_csv(ARQ_OUT, sep=";")
    logging.info(f"Ranking carregado: {ARQ_OUT} | shape={df_rank.shape}")
    display(df_rank.head(10))

    # Histograma de probabilidades
    if "Probabilidade_pct" in df_rank.columns:
        plt.figure(figsize=(7, 5))
        sns.histplot(df_rank["Probabilidade_pct"].dropna(), bins=20, kde=True, color="blue")
        plt.xlabel("Probabilidade Prevista (%)")
        plt.ylabel("Frequência")
        plt.title("Distribuição de Probabilidades (Ranking Exportado)")
        plt.tight_layout()
        plt.show()

    # Barras Top-N primeiro local
    if {"escopo", "cluster_id", "Dim_Produtos.GRUPO_CHILLI", "Probabilidade_pct"} <= set(df_rank.columns):
        first_key = next(iter(df_rank.groupby(["escopo", "cluster_id"]).groups.keys()))
        amostra = df_rank.set_index(["escopo", "cluster_id"]).loc[first_key].reset_index(drop=True)
        plt.figure(figsize=(8, 5))
        sns.barplot(x="Probabilidade_pct", y="Dim_Produtos.GRUPO_CHILLI", data=amostra, palette="viridis")
        plt.title(f"Top-N — {first_key}")
        plt.xlabel("Probabilidade (%)")
        plt.ylabel("Grupo")
        plt.tight_layout()
        plt.show()

    # Heatmap
    if {"escopo", "cluster_id", "Dim_Produtos.GRUPO_CHILLI", "Probabilidade_pct"} <= set(df_rank.columns):
        keys = list(df_rank.groupby(["escopo", "cluster_id"]).groups.keys())[:8]
        sub = df_rank.set_index(["escopo", "cluster_id"]).loc[keys].reset_index()
        sub["local"] = sub["escopo"].astype(str) + " | " + sub["cluster_id"].astype(str)
        piv = sub.pivot_table(index="local", columns="Dim_Produtos.GRUPO_CHILLI",
                              values="Probabilidade_pct", aggfunc="max")
        plt.figure(figsize=(max(8, 0.35 * piv.shape[1] + 3), max(5, 0.35 * piv.shape[0] + 2)))
        sns.heatmap(piv.fillna(0), annot=True, cmap="YlGnBu", cbar_kws={'label': 'Probabilidade (%)'})
        plt.title("Heatmap — Probabilidade por Local × Grupo (Amostra)")
        plt.tight_layout()
        plt.show()

    # Tabela ranqueada
    cols_print = [c for c in [
        "escopo", "cluster_id", "Dim_Lojas.REGIAO_CHILLI",
        "Dim_Produtos.GRUPO_CHILLI", "Dim_Produtos.Grupo_Produto",
        "Preco_Varejo_med", "Probabilidade_pct", "Previsao"
    ] if c in df_rank.columns]
    print("\n==================== RANKING (Amostra Tabular Ordenada) ====================")
    for (esc, clu), df_loc in df_rank.groupby(["escopo", "cluster_id"]):
        print(f"\n>>> Local: escopo={esc} | cluster_id={clu}")
        df_show = df_loc.sort_values("Probabilidade_pct", ascending=False)[cols_print].copy()
        print(df_show.head(12).to_string(index=False))

# Checagens finais
nulls = X_all.isna().sum().sort_values(ascending=False)
logging.info("Nulos nas features (top 10):")
print(nulls.head(10).to_string())

if "Dim_Lojas.REGIAO_CHILLI" in novas.columns:
    regs_novas = set(novas["Dim_Lojas.REGIAO_CHILLI"].dropna().unique())
    regs_hist = set(agg_reg["Dim_Lojas.REGIAO_CHILLI"].unique())
    faltando = regs_novas - regs_hist
    if faltando:
        logging.warning(f"Regiões nas novas SEM histórico: {faltando}")
    else:
        logging.info("Todas as regiões das novas têm histórico.")
